In [1]:
import json
import statistics
import time
from collections import defaultdict

import numpy as np
from sklearn.metrics import adjusted_rand_score, rand_score

## 1. Load data

In [2]:
# Path to your file
file_path = "/home/zeneto/projects/kg4ai-community_detection/data/wiki_categories/nq-train-all.jsonl"

# Dictionary to store categories → list of titles
category_dict = defaultdict(list)

# Read line by line
with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        # Skip empty lines
        line = line.strip()
        if not line:
            continue

        # Parse JSON line
        data = json.loads(line)

        # Extract title and categorias
        title = data.get("title")
        categorias = data.get("categorias", [])

        # Add title under each category
        for cat in categorias:
            category_dict[cat].append(title)

# Optionally convert defaultdict to normal dict
category_dict = dict(category_dict)

# Example: print one category
print("Example category:", next(iter(category_dict.keys())))
print("Articles:", category_dict[next(iter(category_dict.keys()))][:5])  # show first 5


Example category: 2017 American television seasons
Articles: ['The_Walking_Dead_(season_8)', 'Grey%27s_Anatomy_(season_14)', 'The_Walking_Dead_(season_7)', 'America%27s_Got_Talent_(season_12)', 'Grey%27s_Anatomy_(season_14)']


## 2. Basic info


In [3]:
# --- Basic info ---
total_categories = len(category_dict)
print(f"Total number of unique categories: {total_categories}")

# --- Count how many titles per category ---
category_counts = {cat: len(titles) for cat, titles in category_dict.items()}

# --- Find category extremes ---
most_common_cat = max(category_counts, key=category_counts.get)
least_common_cat = min(category_counts, key=category_counts.get)
print(f"Category with the most titles: {most_common_cat} ({category_counts[most_common_cat]} articles)")
print(f"Category with the fewest titles: {least_common_cat} ({category_counts[least_common_cat]} articles)")

# --- Calculate averages and dispersion for category_counts ---
avg_titles_per_category = statistics.mean(category_counts.values())
median_titles_per_category = statistics.median(category_counts.values())
stdev_titles_per_category = statistics.pstdev(category_counts.values())

print(f"\nAverage titles per category: {avg_titles_per_category:.2f}")
print(f"Median titles per category: {median_titles_per_category}")
print(f"Standard deviation (titles per category): {stdev_titles_per_category:.2f}")

# --- Sort categories by article count ---
sorted_cats = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 categories by number of articles:")
for i, (cat, count) in enumerate(sorted_cats[:10]):
    print(f"{i + 1}. {cat}: {count}")

Total number of unique categories: 217562
Category with the most titles: English-language films (17020 articles)
Category with the fewest titles: Computer-assisted surgery (1 articles)

Average titles per category: 11.81
Median titles per category: 3.0
Standard deviation (titles per category): 78.00

Top 10 categories by number of articles:
1. English-language films: 17020
2. American films: 15164
3. Living people: 12082
4. English-language television programs: 10296
5. Billboard Hot 100 number-one singles: 3956
6. UK Singles Chart number-one singles: 3533
7. IMAX films: 3147
8. American male film actors: 3104
9. American male television actors: 2958
10. American television actresses: 2724


In [4]:
# --- Example: Get all titles from a specific category ---
query = "Greek goddesses"  # change this to any category you want
if query in category_dict:
    print(f"Articles under '{query}':")
    for title in category_dict[query]:  # show first 10 titles
        print("-", title)
else:
    print(f"\nCategory '{query}' not found.")

Articles under 'Greek goddesses':
- Persephone
- Theia
- Rhea_(mythology)
- List_of_Greek_mythological_figures
- Persephone
- List_of_Greek_mythological_figures
- Hera
- Hera
- List_of_Greek_mythological_figures
- List_of_Greek_mythological_figures
- Aphrodite
- Gaia
- Selene
- Aphrodite
- List_of_Greek_mythological_figures
- List_of_Greek_mythological_figures
- List_of_Greek_mythological_figures
- Aphrodite
- Hera
- Persephone
- List_of_Greek_mythological_figures
- Gaia_(mythology)
- Moirai
- Circe
- Hecate
- Charites
- Eileithyia
- Circe
- Nike_(mythology)
- List_of_Greek_mythological_figures
- Charites
- List_of_Greek_mythological_figures
- List_of_Greek_mythological_figures
- Hera


In [5]:
# --- Reverse mapping: count how many categories each title belongs to ---
title_to_categories = defaultdict(set)
for cat, titles in category_dict.items():
    for title in titles:
        title_to_categories[title].add(cat)

title_category_counts = {title: len(cats) for title, cats in title_to_categories.items()}

# --- Find title extremes ---
max_cats = max(title_category_counts.values())
min_cats = min(title_category_counts.values())
most_categorized_titles = [t for t, c in title_category_counts.items() if c == max_cats]
least_categorized_titles = [t for t, c in title_category_counts.items() if c == min_cats]

print(f"Title(s) with the most categories ({max_cats}):")
for t in most_categorized_titles:
    print("-", t)

print(f"\nTitle(s) with the fewest categories ({min_cats}):")
for t in least_categorized_titles[:10]:  # limit output
    print("-", t)

# --- Calculate averages and dispersion for title_category_counts ---
avg_categories_per_title = statistics.mean(title_category_counts.values())
median_categories_per_title = statistics.median(title_category_counts.values())
stdev_categories_per_title = statistics.pstdev(title_category_counts.values())

print(f"\nAverage categories per title: {avg_categories_per_title:.2f}")
print(f"Median categories per title: {median_categories_per_title}")
print(f"Standard deviation (categories per title): {stdev_categories_per_title:.2f}")

# --- Optional: show top 10 titles by number of categories ---
sorted_titles = sorted(title_category_counts.items(), key=lambda x: x[1], reverse=True)
print("\nTop 10 titles by number of categories:")
for i, (title, count) in enumerate(sorted_titles[:10]):
    print(f"{i + 1}. {title}: {count}")


Title(s) with the most categories (244):
- Fourth_Geneva_Convention

Title(s) with the fewest categories (1):
- Sleepy_Hollow_(season_4)
- The_Night_Shift_(season_4)
- Suits_(season_7)
- Switched_at_Birth_(season_5)
- Sense8_(season_2)
- Reign_(season_4)
- MasterChef_Junior_(U.S._season_5)
- Xerosere
- Environmental_data
- List_of_members_of_the_House_of_Representatives_of_Nigeria,_2015%E2%80%93present

Average categories per title: 7.33
Median categories per title: 5
Standard deviation (categories per title): 8.42

Top 10 titles by number of categories:
1. Fourth_Geneva_Convention: 244
2. First_Geneva_Convention: 242
3. Third_Geneva_Convention: 242
4. Montreal_Protocol: 232
5. Vienna_Convention_for_the_Protection_of_the_Ozone_Layer: 232
6. Convention_on_Biological_Diversity: 230
7. Convention_on_the_Rights_of_the_Child: 225
8. International_Convention_on_the_Elimination_of_All_Forms_of_Racial_Discrimination: 223
9. Chemical_Weapons_Convention: 221
10. Convention_on_the_Elimination_of_

In [6]:
# Get all titles from all categories
all_titles = []

for titles in category_dict.values():
    all_titles.extend(titles)

# Convert to a set to get only unique ones
unique_titles = set(all_titles)

# Count them
print(f"Total number of unique article titles: {len(unique_titles)}")

# Optionally, see a few examples
print("Example titles:", list(unique_titles)[:10])


Total number of unique article titles: 108013
Example titles: ['Cash_Explosion', 'May_3', 'Scanf_format_string', 'Trelawny_Parish', 'Marcus_Chong', 'Street_Fighter:_The_Legend_of_Chun-Li', 'Income_inequality_in_India', 'My_Two_Dads', 'Petroleum', 'Airfoil']


## 3. Match Category and Articles' ID

In [7]:
import pandas as pd

# 1. Ler o CSV
df = pd.read_csv("/home/zeneto/projects/kg4ai-community_detection/data/v0.0/df_nq_version0.csv")

# 2. Create mapping {encoded_title → id}
title_to_id = dict(zip(df["d_properties_title_encode"], df["id"]))

# Detect duplicates in CSV
if df["d_properties_title_encode"].duplicated().any():
    print("⚠️ Warning: There are duplicate encoded titles in your CSV!")
    duplicates = df[df["d_properties_title_encode"].duplicated(keep=False)]
    print(duplicates)

# 3. Build the new dictionary + track missing
category_dict_id = {}
missing_titles = set()
matched_count = 0
total_titles = 0
category_id_counter = 0  # Initialize the counter for category IDs

for category, titles in category_dict.items():
    ids = []
    for title in titles:
        total_titles += 1
        if title in title_to_id:
            ids.append(title_to_id[title])
            matched_count += 1
        else:
            missing_titles.add(title)
    # 4. Assign Category ID
    if ids:
        # Create the category ID (e.g., 'cat_0', 'cat_1', ...)
        category_id = f"cat_{category_id_counter}"
        category_dict_id[category_id] = ids
        category_id_counter += 1

# --- Summary Report ---
print(f"✅ Successfully matched: {matched_count}/{total_titles} titles ({matched_count/total_titles:.2%})")
print(f"❌ Missing titles: {len(missing_titles)}")
print(f"Total categories with assigned IDs: {category_id_counter}")

# --- Optionally show some missing examples ---
if missing_titles:
    print("\nExamples of missing titles:")
    for t in list(missing_titles)[:10]:
        print("-", t)


✅ Successfully matched: 2569240/2569242 titles (100.00%)
❌ Missing titles: 1
Total categories with assigned IDs: 217562

Examples of missing titles:
- NaN


## 4. Load Community

In [8]:
# Load the JSON file containing community assignments
json_path = "/home/zeneto/projects/kg4ai-community_detection/community_detection/output/run142-NQv0-best_run/run142_raw_communities.json"

with open(json_path, 'r') as f:
    communities_data = json.load(f)

print("Available algorithms:", list(communities_data.keys()))
for algo, communities in communities_data.items():
    print(f"{algo}: {len(communities)} communities")

# Select the algorithm with the best Modularity
selected_algorithm = "leiden"
communities = communities_data.get(selected_algorithm, [])

Available algorithms: ['infomap', 'louvain', 'leiden', 'k-means']
infomap: 1249 communities
louvain: 24 communities
leiden: 19 communities
k-means: 40 communities


### Check IDs intersection

In [9]:
# --- Extract unique IDs from category_dict_id ---
category_ids = set()
for ids in category_dict_id.values():
    category_ids.update(ids)

print(f"Total unique IDs in category_dict_id: {len(category_ids):,}")

# --- Extract unique IDs from communities ---
community_ids = set()
for community in communities:
    community_ids.update(community)

print(f"Total unique IDs in communities: {len(community_ids):,}")

# --- Compare ---
in_both = category_ids & community_ids
only_in_category = category_ids - community_ids
only_in_communities = community_ids - category_ids

print(f"\n✅ IDs in both: {len(in_both):,}")
print(f"⚠️ IDs only in category_dict_id: {len(only_in_category):,}")
print(f"⚠️ IDs only in communities: {len(only_in_communities):,}")

# --- Show some examples if mismatches exist ---
if only_in_category:
    print("\nExamples of IDs only in category_dict_id:")
    print(list(only_in_category)[:10])

if only_in_communities:
    print("\nExamples of IDs only in communities:")
    print(list(only_in_communities)[:10])


Total unique IDs in category_dict_id: 108,012
Total unique IDs in communities: 107,534

✅ IDs in both: 107,481
⚠️ IDs only in category_dict_id: 531
⚠️ IDs only in communities: 53

Examples of IDs only in category_dict_id:
[40963, 26628, 53260, 67604, 38936, 92184, 10267, 14367, 63520, 86048]

Examples of IDs only in communities:
[100867, 68487, 41096, 87816, 98699, 89234, 92052, 99860, 34710, 36503]


### Check Categories from Title

In [10]:
# Find title on df
title = "Fourth_Geneva_Convention"
df[df["d_properties_title_encode"] == title]

,id,d_properties_document_title,d_properties_title_encode,cites_ids
70509,72150,Fourth Geneva Convention,Fourth_Geneva_Convention,"[14089, 25290, 36231, 38878, 13135, 5924, 6530..."


In [11]:
# Find id on df
id = 213
print(df[df["id"] == id])

# Find title on categories
title = df[df["id"] == id]["d_properties_title_encode"].values[0]
cat_from_title = []
for cat, titles in category_dict.items():
    if title in titles:
        cat_from_title.append(cat)

# Show categories found
print(f"\nTitle '{title}' is in categories:")
for cat in cat_from_title:
    print("-", cat)


      id d_properties_document_title d_properties_title_encode  \
161  213              Paleomagnetism            Paleomagnetism   

                                             cites_ids  
161  [1697, 2758, 2723, 1124, 6036, 3414, 2996, 423...  

Title 'Paleomagnetism' is in categories:
- Historical geology
- Geochronological dating methods
- Paleomagnetism


## 5. Calculate Metrics

Context: Ground Truth vs. Predicted Communities

**Label Mappings**
- **Ground truth (X):** Each article ID is mapped to one or more Wikipedia categories (`id_to_cat_multi_label`).
- **Predicted (Y):** Each article ID is mapped to a single community label detected by **Leiden** (`id_to_comm`).

**Ground Truth Overview - Wikipedia Article Categories**

* **Total categories:** `217,562`
* **Average categories per article:** `7.33`
* **Most frequent category:** *English-language films* (`17,020` articles)
* **Article with most categories:** *Fourth Geneva Convention* (`244` categories)

**Prediction Overview - Leiden Community Detection**

* **Algorithm:** Leiden
* **Modularity:** ≈ `0.6`
* **Detected communities:** `19`
* **Biggest Community:** *C0 - Film, Series, Season, Episodes* (`20,754` articles)

**Summary:**

We aligned **Wikipedia article categories** (multi-label ground truth) with **Leiden-predicted communities** (single-label prediction).


In [12]:
# --- Extract unique IDs from both sources ---
category_ids = set()
for ids in category_dict_id.values():
    category_ids.update(ids)

community_ids = set()
for comm in communities:
    community_ids.update(comm)

# --- Keep only IDs that appear in both ---
shared_ids = category_ids & community_ids
print(f"✅ Shared IDs: {len(shared_ids):,}")

✅ Shared IDs: 107,481


In [13]:
# --- Build label mappings ---

# Map each ID to its category label (X) - "Ground Truth"
id_to_cat_multi_label  = defaultdict(set)
for cat_id_string, ids in category_dict_id.items():
    for i in ids:
        if i in shared_ids:
            id_to_cat_multi_label [i].add(cat_id_string)

# Map each ID to its community label (Y) - "Prediction"
id_to_comm = {}
for comm_idx, ids in enumerate(communities):
    for i in ids:
        if i in shared_ids:
            id_to_comm[i] = comm_idx

### Simplify Ground Truth (X) to Single-Label using Dominant Category

In [14]:
# Precompute category sizes
category_sizes = {
    cat_id_string: len(ids)
    for cat_id_string, ids in category_dict_id.items()
}   

id_to_cat_single_label = {}
for i in shared_ids:
    # Get the set of categories this shared ID belongs to
    assigned_cats = id_to_cat_multi_label.get(i)

    if len(assigned_cats) == 1:
        # If no overlap, simply use the single category ID string
        id_to_cat_single_label[i] = list(assigned_cats)[0]
    else:
        # --- DOMINANT CATEGORY SELECTION LOGIC ---
        best_cat = None
        max_size = -1
        
        # Iterate over all categories the ID belongs to
        for cat_id in assigned_cats:
            size = category_sizes[cat_id]
            
            # If the current category is larger than the best found so far
            if size > max_size:
                max_size = size
                best_cat = cat_id
            # Tie-breaking rule: if sizes are equal, use the alphabetically first cat_id
            elif size == max_size and cat_id < best_cat:
                 best_cat = cat_id
        
        id_to_cat_single_label[i] = best_cat

# --- Align both label vectors ---
shared_ids_list = list(shared_ids)
labels_X = [id_to_cat_single_label[i] for i in shared_ids_list]
labels_Y = [id_to_comm[i] for i in shared_ids_list]

# --- Compute Rand Index ---
ri = rand_score(labels_X, labels_Y)
print(f"Rand Index: {ri:.6f}")

# --- Compute ARI ---
ari = adjusted_rand_score(labels_X, labels_Y)
print(f"Adjusted Rand Index: {ari:.6f}")

Rand Index: 0.894868
Adjusted Rand Index: 0.041288


- A high RI (or ARI) is expected when the number of partitions is small, as is the case here (19 detected communities vs. a simplified ground truth with a likely small number of dominant categories). 
- It primarily indicates that the Leiden algorithm's partitioning is structurally similar to the simplified, dominant categories, but it masks the complexity of the true multi-label ground truth.

- The ARI value being extremely close to zero means the agreement between the detected communities and the ground truth is no better than random chance. 
- When correcting for the high chance agreement introduced by mapping a massive, detailed, multi-label structure (217K categories) down to just 19 partitions, the clustering provides no meaningful, non-random insight. The algorithm is effectively failing to identify or align with the specific, complex structure hidden within the Wikipedia categories.

### Pairwise Co-membership Evaluation

For every pair of IDs `(i, j)`:

| Case           | Ground Truth                                                     | 
| -------------- | ---------------------------------------------------------------- | 
| **True Positive (TP)**  |`(i, j)` **share** ≥1 true group **and** are in **same** predicted group     |            
| **True Negative (TN)**  |`(i, j)` **not** in same predicted group and share **no** true group         |       
| **False Positive (FP)** |`(i, j)` in **same** predicted group but share **no** true group         |            
| **False Negative (FN)** |`(i, j)` **share** ≥1 true group but are **not** in same predicted group |            

- *true group* = same category
- *predicted group* = same community

Then compute **pairwise precision, recall, and F1**:

$$
\text{Rand Index (RI)} = \frac{TP + TN}{TP + FP + FN + TN}
\quad
\text{Precision} = \frac{TP}{TP + FP}
\quad
\text{Recall} = \frac{TP}{TP + FN}
\quad
\text{F1} = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
$$

In [15]:
def calculate_pairwise_metrics(id_to_cat_multi_label, id_to_comm):
    """
    Calculate pairwise co-membership metrics for multi-label true groups and single-label predicted communities.
    
    Args:
        id_to_cat_multi_label: dict mapping ID to set of true categories
        id_to_comm: dict mapping ID to predicted community
    
    Returns:
        dict with metrics and counts
    """
    # Get all IDs
    all_ids = list(id_to_comm.keys())
    n_pairs = len(all_ids) * (len(all_ids) - 1) // 2

    print(f"Total pairs to evaluate: {n_pairs}")
    
    # Initialize counters
    TP = FP = FN = TN = 0
    start_time = time.time()
    
    # Iterate over all unique pairs of IDs
    for i in range(len(all_ids)):
        for j in range(i + 1, len(all_ids)):
            id_i = all_ids[i]
            id_j = all_ids[j]
            
            # Check true group co-membership (share at least one category)
            true_co_membership = len(id_to_cat_multi_label[id_i] & id_to_cat_multi_label[id_j]) > 0
            
            # Check predicted group co-membership (same community)
            pred_co_membership = (id_to_comm[id_i] == id_to_comm[id_j])
            
            # Classify the pair
            if true_co_membership and pred_co_membership:
                TP += 1
            elif (not true_co_membership) and pred_co_membership:
                FP += 1
            elif true_co_membership and (not pred_co_membership):
                FN += 1
            else:  # not true_co_membership and not pred_co_membership
                TN += 1
    
    # Calculate metrics
    rand_index = (TP + TN) / (TP + TN + FP + FN) if (TP + TN + FP + FN) > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'total_pairs': TP + FP + FN + TN,
        'rand_index': rand_index,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'time_seconds': time.time() - start_time
    }

# Calculate metrics
# results_raw = calculate_pairwise_metrics(id_to_cat_multi_label, id_to_comm)

print("Pairwise Co-membership Metrics:")
print(f"True Positives (TP): {results_raw['TP']}")
print(f"False Positives (FP): {results_raw['FP']}")
print(f"False Negatives (FN): {results_raw['FN']}")
print(f"True Negatives (TN): {results_raw['TN']}")
print(f"Total pairs: {results_raw['total_pairs']}")

sum = results_raw['TP'] + results_raw['FP'] + results_raw['FN'] + results_raw['TN']
print(f"\nSum check (should equal total pairs): {sum} == {results_raw['total_pairs']}")
if sum != results_raw['total_pairs']:
    print("⚠️ Warning: Sum of TP, FP, FN, TN does not equal total pairs!")
else:
    print("✅ Sum check passed.")

print(f"\nRand Index: {results_raw['rand_index']:.4f}")
print(f"Precision: {results_raw['precision']:.4f}")
print(f"Recall: {results_raw['recall']:.4f}")
print(f"F1 Score: {results_raw['f1']:.4f}")

print(f"\nComputation Time: {results_raw['time_seconds']:.2f} seconds")

Pairwise Co-membership Metrics:


NameError: name 'results_raw' is not defined

In [ ]:
def calculate_pairwise_metrics_efficient(id_to_cat_multi_label, id_to_comm):
    """
    More efficient version that uses community structure to reduce comparisons.
    """
    # Group IDs by predicted community
    comm_to_ids = defaultdict(list)
    for id_val, comm in id_to_comm.items():
        comm_to_ids[comm].append(id_val)
    
    TP = FP = FN = TN = 0
    start_time = time.time()
    
    # Count pairs within same predicted communities
    for comm, ids_in_comm in comm_to_ids.items():
        n_in_comm = len(ids_in_comm)
        if n_in_comm < 2:
            continue
            
        # Count pairs within this community
        for i in range(n_in_comm):
            for j in range(i + 1, n_in_comm):
                id_i = ids_in_comm[i]
                id_j = ids_in_comm[j]
                
                # Check if they share at least one true category
                if len(id_to_cat_multi_label[id_i] & id_to_cat_multi_label[id_j]) > 0:
                    TP += 1
                else:
                    FP += 1
    
    # Count pairs across different communities (FN and TN)
    all_communities = list(comm_to_ids.keys())
    total_pairs_so_far = TP + FP
    
    # For FN: pairs that share true categories but are in different predicted communities
    # We'll use a different approach for efficiency
    
    # Create a mapping from category to IDs for efficient lookup
    cat_to_ids = defaultdict(set)
    for id_val, cats in id_to_cat_multi_label.items():
        for cat in cats:
            cat_to_ids[cat].add(id_val)
    
    # Count FN: for each category, count pairs that share this category but are in different communities
    fn_counted = set()
    for cat, ids_in_cat in cat_to_ids.items():
        ids_list = list(ids_in_cat)
        for i in range(len(ids_list)):
            for j in range(i + 1, len(ids_list)):
                id_i, id_j = ids_list[i], ids_list[j]
                # Use frozenset to ensure we count each pair only once
                pair = frozenset([id_i, id_j])
                if pair in fn_counted:
                    continue
                    
                # If they're in different predicted communities
                if id_to_comm[id_i] != id_to_comm[id_j]:
                    FN += 1
                    fn_counted.add(pair)
    
    # Total possible pairs
    n_ids = len(id_to_comm)
    total_possible_pairs = n_ids * (n_ids - 1) // 2
    
    # TN = total pairs - (TP + FP + FN)
    TN = total_possible_pairs - (TP + FP + FN)
    
    # Calculate metrics
    rand_index = (TP + TN) / total_possible_pairs if total_possible_pairs > 0 else 0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'TP': TP,
        'FP': FP,
        'FN': FN,
        'TN': TN,
        'total_pairs': total_possible_pairs,
        'time_seconds': time.time() - start_time,
        'rand_index': rand_index,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Calculate metrics
results = calculate_pairwise_metrics_efficient(id_to_cat_multi_label, id_to_comm)

print("Pairwise Co-membership Metrics:")
print(f"True Positives (TP): {results['TP']}")
print(f"False Positives (FP): {results['FP']}")
print(f"False Negatives (FN): {results['FN']}")
print(f"True Negatives (TN): {results['TN']}")
print(f"Total pairs: {results['total_pairs']}")

sum = results['TP'] + results['FP'] + results['FN'] + results['TN']
print(f"\nSum check (should equal total pairs): {sum} == {results['total_pairs']}")
if sum != results['total_pairs']:
    print("⚠️ Warning: Sum of TP, FP, FN, TN does not equal total pairs!")
else:
    print("✅ Sum check passed.")

print(f"\nRand Index: {results['rand_index']:.4f}")
print(f"Precision: {results['precision']:.4f}")
print(f"Recall: {results['recall']:.4f}")
print(f"F1 Score: {results['f1']:.4f}")

print(f"\nComputation Time: {results['time_seconds']:.2f} seconds")

Pairwise Co-membership Metrics:
True Positives (TP): 20370689
False Positives (FP): 590235997
False Negatives (FN): 14028525
True Negatives (TN): 5151393729
Total pairs: 5776028940

Sum check (should equal total pairs): 5776028940 == 5776028940
✅ Sum check passed.

Rand Index: 0.8954
Precision: 0.0334
Recall: 0.5922
F1 Score: 0.0632

Computation Time: 258.80 seconds


#### Pair Counts
| Count | Value | Contextual Interpretation |
| :--- | :--- | :--- |
| **TP** (Same Comm, Shared Cat) | **20.37M** | A decent number of correct intra-community pairs. |
| **FP** (Same Comm, **No Shared Cat**) | **590.24M** | **MASSIVE:** This number is $\mathbf{\approx 29 \times}$ larger than TP. This is the **primary problem**; the Leiden algorithm is combining far too many unrelated articles into the same community. This is why Precision is so low. |
| **FN** (Diff Comm, **Shared Cat**) | **14.03M** | Lower than TP, indicating that while some related articles are incorrectly separated (low Recall), the algorithm is better at keeping related articles together than it is at keeping unrelated articles apart. |
| **TN** (Diff Comm, No Shared Cat) | **5.15B** | **Dominant Factor:** This huge number (over 5 billion) represents the vast majority of all possible pairs. It means most pairs of articles in the dataset share no categories and are correctly separated by Leiden, which inflates the Rand Index. |


#### Metrics

| Metric | Value | Interpretation |
| :--- | :--- | :--- |
| **Rand Index** | **0.8954** | Again, a high RI (similar to the simplified one). This is dominated by the **True Negatives** (TN) and indicates the Leiden algorithm is **very good at separating pairs of articles that have *nothing* in common** (no shared categories). |
| **Precision** | **0.0334** | **Very Low.** When the Leiden algorithm places two articles in the **same community** (Predicted Positive), there is only a **3.34% chance** that those two articles actually share at least one Wikipedia category. This points to **low purity** and significant **False Positives (FP)**. |
| **Recall** | **0.5922** | **Moderately Low.** When two articles truly share a category (True Positive + False Negative), the Leiden algorithm correctly places them in the **same community** (True Positive) **59.22%** of the time. This points to moderate **coverage** of the true clusters. |
| **F1 Score** | **0.0632** | **Very Low.** This is the harmonic mean of Precision and Recall. The extremely low F1-score is primarily dragged down by the low **Precision**, confirming that the **Leiden communities lack purity** relative to the article categories. |

### Pairwise Co-membership Evaluation (Jaccard-weighted)

For every pair of IDs `(i, j)`:

| Case                    | Ground Truth                                                               | Ground Truth Weight |   Sum Component (Weighted) |                                                                                                                                   
| ----------------------- | -------------------------------------------------------------------------- | ------------------------------| --|
| **True Positive (TP)**  | `(i, j)` **share** ≥1 true group **and** are in **same** predicted group         | $$\text{Jaccard}(X_i, X_j)$$ | $$\sum_{\text{TP}} \text{Jaccard}(X_i, X_j)$$ |
| **True Negative (TN)**  | `(i, j)` **not** in same predicted group and share **no** true group | $$1 - \text{Jaccard}(X_i, X_j)$$ |$$\sum_{\text{TN}} (1 - 0) = TN$$ |               
| **False Positive (FP)**  | `(i, j)` in **same** predicted group but share **no** true group |$$1 - \text{Jaccard}(X_i, X_j)$$ |$$\sum_{\text{TN}} (1 - 0) = FP$$ |    
| **False Negative (FN)**  | `(i, j)` **share** ≥1 true group but are **not** in same predicted group | $$\text{Jaccard}(X_i, X_j)$$ | $$\sum_{\text{TP}} \text{Jaccard}(X_i, X_j)$$ |

- *true group* = same category
- *predicted group* = same community

Then compute a **Weighted Rand Index (RI)**, defined as:

$$
\text{WRI} = \frac{Weighted Agreement}{\text{Total number of pairs}}
$$

Where:

$$\text{Weighted Agreement} = \underbrace{\sum_{\text{TP}} \text{Jaccard}(X_i, X_j)}_{\text{Weighted TP}} + \underbrace{TN}_{\text{Weighted TN}}$$

In [ ]:
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0


def calculate_pairwise_metrics_weighted(id_to_cat_multi_label, id_to_comm):
    """
    More efficient version that uses community structure to reduce comparisons.
    """
    # Group IDs by predicted community
    comm_to_ids = defaultdict(list)
    for id_val, comm in id_to_comm.items():
        comm_to_ids[comm].append(id_val)
    
    TP_weighted = TP_count = TN_count_and_weighted = FP_count_and_weighted = FN_weighted = FN_count = 0
    start_time = time.time()
    
    # Count pairs within same predicted communities
    for comm, ids_in_comm in comm_to_ids.items():
        n_in_comm = len(ids_in_comm)
        if n_in_comm < 2:
            continue
            
        # Count pairs within this community
        for i in range(n_in_comm):
            for j in range(i + 1, n_in_comm):
                id_i = ids_in_comm[i]
                id_j = ids_in_comm[j]
                
                # Calculate Jaccard similarity
                weight = jaccard_similarity(id_to_cat_multi_label[id_i], id_to_cat_multi_label[id_j])

                # Check if they share at least one true category
                if len(id_to_cat_multi_label[id_i] & id_to_cat_multi_label[id_j]) > 0:
                    TP_weighted += weight
                    TP_count += 1
                else:
                    # FP += 1 - weight, but we count FP as 1 since Jaccard weight is 0
                    FP_count_and_weighted += 1
    
    # Create a mapping from category to IDs for efficient lookup
    cat_to_ids = defaultdict(set)
    for id_val, cats in id_to_cat_multi_label.items():
        for cat in cats:
            cat_to_ids[cat].add(id_val)
    
    # Count FN: for each category, count pairs that share this category but are in different communities
    fn_counted = set()
    for cat, ids_in_cat in cat_to_ids.items():
        ids_list = list(ids_in_cat)
        for i in range(len(ids_list)):
            for j in range(i + 1, len(ids_list)):
                id_i, id_j = ids_list[i], ids_list[j]

                # Use frozenset to ensure we count each pair only once
                pair = frozenset([id_i, id_j])
                if pair in fn_counted:
                    continue
                    
                # If they're in different predicted communities
                if id_to_comm[id_i] != id_to_comm[id_j]:
                    # Calculate Jaccard similarity
                    weight = jaccard_similarity(id_to_cat_multi_label[id_i], id_to_cat_multi_label[id_j])
                    FN_weighted += weight
                    FN_count += 1
                    fn_counted.add(pair)
    
    # Total possible pairs
    n_ids = len(id_to_comm)
    total_possible_pairs = n_ids * (n_ids - 1) // 2

    # True Negatives (TN) - TN_count is not needed for weighting, since Jaccard weight is 0
    TN_count_and_weighted = total_possible_pairs - (TP_count + FP_count_and_weighted + FN_count)

    # Total weighted pairs
    total_weighted_pairs = TP_weighted + FP_count_and_weighted + FN_weighted + TN_count_and_weighted
    
    # Calculate metrics
    rand_index = (TP_weighted + TN_count_and_weighted) / total_weighted_pairs if total_weighted_pairs > 0 else 0
    precision = TP_weighted / (TP_weighted + FP_count_and_weighted) if (TP_weighted + FP_count_and_weighted) > 0 else 0
    recall = TP_weighted / (TP_weighted + FN_weighted) if (TP_weighted + FN_weighted) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'TP': TP_weighted,
        'FP': FP_count_and_weighted,
        'FN': FN_weighted,
        'TN': TN_count_and_weighted,
        'total_pairs': total_possible_pairs,
        'total_weighted_pairs': total_weighted_pairs,
        'time_seconds': time.time() - start_time,
        'rand_index': rand_index,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Calculate metrics
results_weighted = calculate_pairwise_metrics_weighted(id_to_cat_multi_label, id_to_comm)

print("Pairwise Co-membership Metrics:")
print(f"True Positives (TP): {results_weighted['TP']}")
print(f"True Negatives (TN): {results_weighted['TN']}")
print(f"False Positives (FP): {results_weighted['FP']}")
print(f"False Negatives (FN): {results_weighted['FN']}")
print(f"Total pairs: {results_weighted['total_pairs']}")
print(f"Total weighted pairs: {results_weighted['total_weighted_pairs']}")

sum = results_weighted['TP'] + results_weighted['FP'] + results_weighted['FN'] + results_weighted['TN']
print(f"\nSum check (should equal total pairs): {sum} == {results_weighted['total_pairs']}")
if sum != results_weighted['total_pairs']:
    print("⚠️ Warning: Sum of TP, FP, FN, TN does not equal total pairs!")
else:
    print("✅ Sum check passed.")

print(f"\nRand Index: {results_weighted['rand_index']:.4f}")
print(f"Precision: {results_weighted['precision']:.4f}")
print(f"Recall: {results_weighted['recall']:.4f}")
print(f"F1 Score: {results_weighted['f1']:.4f}")

print(f"\nComputation Time: {results_weighted['time_seconds']:.2f} seconds")

Pairwise Co-membership Metrics:
True Positives (TP): 1402211.4217309589
True Negatives (TN): 5151393729
False Positives (FP): 590235997
False Negatives (FN): 725902.3807737853
Total pairs: 5776028940
Total weighted pairs: 5743757839.802505

Sum check (should equal total pairs): 5743757839.802505 == 5776028940
⚠️ Warning: Sum of TP, FP, FN, TN does not equal total pairs!

Rand Index: 0.8971
Precision: 0.0024
Recall: 0.6589
F1 Score: 0.0047

Computation Time: 675.70 seconds


In [ ]:
tp_weighted = results_weighted['TP']
tp_raw = 20370689

tn_weighted = results_weighted['TN']
tn_raw = 5151393729

fp_weighted = results_weighted['FP']
fp_raw = 590235997

fn_weighted = results_weighted['FN']
fn_raw = 14028525

# Calculate differences
tp_diff = tp_weighted - tp_raw
tn_diff = tn_weighted - tn_raw
fp_diff = fp_weighted - fp_raw
fn_diff = fn_weighted - fn_raw

print(f"TP difference: {tp_diff} ({(tp_diff / tp_raw) * 100:.6f}%)")
print(f"TN difference: {tn_diff} ({(tn_diff / tn_raw) * 100:.6f}%)")
print(f"FP difference: {fp_diff} ({(fp_diff / fp_raw) * 100:.6f}%)")
print(f"FN difference: {fn_diff} ({(fn_diff / fn_raw) * 100:.6f}%)")

# Calculate sums differences
sum_weighted = tp_weighted + tn_weighted + fp_weighted + fn_weighted
sum_raw = tp_raw + tn_raw + fp_raw + fn_raw
sum_diff = sum_weighted - sum_raw
print(f"Total pairs difference: {sum_diff} ({(sum_diff / sum_raw) * 100:.6f}%)")

sum_shared_cats_metrics_weighted = tp_weighted + fn_weighted
sum_shared_cats_metrics_raw = tp_raw + fn_raw
sum_shared_cats_metrics_diff = sum_shared_cats_metrics_weighted - sum_shared_cats_metrics_raw
print(f"Sum (TP + FN) difference: {sum_shared_cats_metrics_diff} ({(sum_shared_cats_metrics_diff / sum_shared_cats_metrics_raw) * 100:.6f}%)")

# Calculate Rand Index differences
ri_weighted = results_weighted['rand_index']
ri_raw =  0.8954
ri_diff = ri_weighted - ri_raw
print(f"Rand Index difference: {ri_diff} ({(ri_diff / ri_raw) * 100:.6f}%)")

TP difference: -18968477.578269042 (-93.116524%)
TN difference: 0 (0.000000%)
FP difference: 0 (0.000000%)
FN difference: -13302622.619226215 (-94.825526%)
Total pairs difference: -32271100.19749546 (-0.558707%)
Sum (TP + FN) difference: -32271100.197495256 (-93.813481%)
Rand Index difference: 0.0017123233494305001 (0.191236%)


### Jaccard-Based Similarity Score

| **Component**                            | **Definition**                                                                   | **Notes**                                                                  |
| ---------------------------------------- | -------------------------------------------------------------------------------- | -------------------------------------------------------------------------- |
| **Ground Truth Sets**                    | *C*<sub>i</sub>: Set of true categories for article *i*.                               | The Ground Truth is **multi-label**.                                       |
| **Jaccard Index (J)**                    | $$J(C_i, C_j) = \dfrac{\lvert C_i \cap C_j \rvert}{\lvert C_i \cup C_j \rvert}$$ | Measures the overlap between category sets *C*<sub>i</sub> and *C*<sub>j</sub>.            |
| **Same Pairs (P<sub>same</sub>)** | Pairs `(i, j)` that are in the same predicted community.                 | Includes both **TP** (truly grouped) and **FP** (falsely grouped) pairs. |
| **Separated Pairs (P<sub>sep</sub>)** | Pairs `(i, j)` that are **not** in the same predicted community.                 | Includes both **TN** (truly separate) and **FN** (falsely separate) pairs. |


New overall evaluation metric, with a custom $\frac{a+b}{\text{total}}$, is defined as:

$$\mathbf{Custom\ Similarity} = \frac{\mathbf{a} + \mathbf{b}}{\text{Total Number of Pairs}} = \frac{(\text{Quality of Grouping}) + (\text{Quality of Separation})}{\text{Total Possible Decisions}}$$


Where:

- $\mathbf{a}$ = $\sum_{(i, j) \in \text{P}_{\text{Same}}} \left (J(C_i, C_j)\right)$.

- $\mathbf{b}$ = $\sum_{(i, j) \in \text{P}_{\text{Sep}}} \left(1 - J(C_i, C_j)\right)$.

- $\text{Total Number of Pairs} = Every\ possible\ pair\ of\ items = \frac{n(n-1)}{2}$

The term $\mathbf{a}$ quantifies the **quality of the grouping** for all pairs placed in the **same** predicted community ($\mathbf{P}_{\text{Same}}$). Since $J \in [0, 1]$:

- If $J \approx 0$, the categories are completely disjoint, so $a \approx 0$. The algorithm is correctly penalized (receives zero reward) for falsely grouping dissimilar pairs.

- If $J \approx 1$, the categories are nearly identical, so $a \approx 1$. The algorithm is highly rewarded for correctly grouping similar pairs.

The term $\mathbf{b}$ quantifies the **quality of the separation** for all pairs placed in a **different** predicted community ($\mathbf{P}_{\text{Sep}}$). Since $J \in [0, 1]$:

- If $J \approx 0$, the categories are completely disjoint, so $1-J \approx 1$. The algorithm is highly rewarded for separating this pair.

- If $J \approx 1$, the categories are nearly identical, so $1-J \approx 0$. The algorithm is penalized (receives no score) for separating this pair, as they should have been together ($\text{FN}$).

In [ ]:
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union > 0 else 0

def calculate_jaccard_custom_similarity(id_to_cat_multi_label, id_to_comm):
    """
    Calculate pairwise co-membership metrics for multi-label true groups and single-label predicted communities.
    
    Args:
        id_to_cat_multi_label: dict mapping ID to set of true categories
        id_to_comm: dict mapping ID to predicted community
    
    Returns:
        dict with metrics and counts
    """
    # Get all IDs
    all_ids = list(id_to_comm.keys())
    total_possible_pairs = len(all_ids) * (len(all_ids) - 1) // 2

    print(f"Total pairs to evaluate: {total_possible_pairs}")
    
    # Initialize counters
    a = b = 0
    start_time = time.time()
    
    # Iterate over all unique pairs of IDs
    for i in range(len(all_ids)):
        for j in range(i + 1, len(all_ids)):
            id_i = all_ids[i]
            id_j = all_ids[j]
            
            # Check predicted group co-membership (same community)
            pred_co_membership = (id_to_comm[id_i] == id_to_comm[id_j])

            # Calculate Jaccard similarity
            weight = jaccard_similarity(id_to_cat_multi_label[id_i], id_to_cat_multi_label[id_j])
            
            # Classify the pair
            if pred_co_membership:
                a += weight
            else:
                b += (1 - weight)
    
    # Total sum of weights
    total_weighted_pairs = a + b

    # Jaccard Based Similarity Score
    jaccard_similarity_score = total_weighted_pairs / total_possible_pairs if total_possible_pairs > 0 else 0
    
    return {
        'total_possible_pairs': total_possible_pairs,
        'a_quality_grouping': a,
        'b_quality_separation': b,
        'total_weighted_pairs': total_weighted_pairs,
        'jaccard_similarity_score': jaccard_similarity_score,
        'time_seconds': time.time() - start_time
    }

In [ ]:
# Calculate metrics
results_jaccard = calculate_jaccard_custom_similarity(id_to_cat_multi_label, id_to_comm)

print("Pairwise Co-membership Metrics:")
print(f"Quality of Grouping (a): {results_jaccard['a_quality_grouping']}")
print(f"Quality of Separation (b): {results_jaccard['b_quality_separation']}")
print(f"Total possible pairs: {results_jaccard['total_possible_pairs']}")
print(f"Total weighted pairs: {results_jaccard['total_weighted_pairs']}")

print(f"\nJaccard Similarity Score: {results_jaccard['jaccard_similarity_score']:.4f}")

print(f"\nComputation Time: {results_jaccard['time_seconds']:.2f} seconds")

Total pairs to evaluate: 5776028940
Pairwise Co-membership Metrics:
Quality of Grouping (a): 1402211.4217309589
Quality of Separation (b): 5164696351.643766
Total possible pairs: 5776028940
Total weighted pairs: 5166098563.065497

Jaccard Similarity Score: 0.8944

Computation Time: 5790.86 seconds


### Normalized Jaccard-Based Similarity

We'll define the new metrics by first counting the number of pairs that contribute to the $\mathbf{a}$ and $\mathbf{b}$ terms, which are $\mathbf{P}_{\text{Same}}$ and $\mathbf{P}_{\text{Sep}}$, respectively.

**1. Pair Counts**

| **Component**                            | **Definition**                                                                   | **Formula**                                                                  |
| :--------------------------------------- | :------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| **Same Pair Count ($\mathbf{N}_{\text{Same}}$)** | The total number of pairs $(i, j)$ in the same predicted community ($\mathbf{P}_{\text{Same}}$). | $$\mathbf{N}_{\text{Same}} = \lvert \mathbf{P}_{\text{Same}} \rvert$$                                                         |
| **Separated Pair Count ($\mathbf{N}_{\text{Sep}}$)** | The total number of pairs $(i, j)$ **not** in the same predicted community ($\mathbf{P}_{\text{Sep}}$). | $$\mathbf{N}_{\text{Sep}} = \lvert \mathbf{P}_{\text{Sep}} \rvert$$                                                         |

**Note**: The sum of these counts must equal the total number of pairs:
$$\mathbf{N}_{\text{Same}} + \mathbf{N}_{\text{Sep}} = \frac{n(n-1)}{2}$$

---

**2. Average Grouping Quality ($\mathbf{A_G}$)**

This metric normalizes the total grouping quality ($\mathbf{a}$) by the number of pairs that were grouped ($\mathbf{N}_{\text{Same}}$). It represents the **average Jaccard Index** for all pairs placed together.

| **Component**                            | **Definition**                                                                   | **Formula**                                                                  |
| :--------------------------------------- | :------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| **Average Grouping Quality ($\mathbf{A_G}$)** | The average Jaccard Index for all $\mathbf{P}_{\text{Same}}$ pairs.                                             | $$\mathbf{A_G} = \frac{\mathbf{a}}{\mathbf{N}_{\text{Same}}} = \frac{1}{\mathbf{N}_{\text{Same}}} \sum_{(i, j) \in \mathbf{P}_{\text{Same}}} J(C_i, C_j)$$ |

**Interpretation**:
- $A_G \approx 1$ means the algorithm's groupings are highly accurate; grouped pairs have very similar true categories.
- $A_G \approx 0$ means the algorithm is often grouping completely dissimilar pairs.

---

**3. Average Separation Quality ($\mathbf{A_S}$)**

This metric normalizes the total separation quality ($\mathbf{b}$) by the number of pairs that were separated ($\mathbf{N}_{\text{Sep}}$). It represents the **average dissimilarity ($1-J$)** for all pairs placed in different communities.

| **Component**                            | **Definition**                                                                   | **Formula**                                                                  |
| :--------------------------------------- | :------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| **Average Separation Quality ($\mathbf{A_S}$)** | The average dissimilarity $(1-J)$ for all $\mathbf{P}_{\text{Sep}}$ pairs.                                             | $$\mathbf{A_S} = \frac{\mathbf{b}}{\mathbf{N}_{\text{Sep}}} = \frac{1}{\mathbf{N}_{\text{Sep}}} \sum_{(i, j) \in \mathbf{P}_{\text{Sep}}} \left(1 - J(C_i, C_j)\right)$$ |

**Interpretation**:
- $A_S \approx 1$ means the algorithm's separations are highly accurate; separated pairs have very dissimilar true categories.
- $A_S \approx 0$ means the algorithm is often separating pairs that are highly similar (many False Negatives).

---

**4. Jaccard-Based Average Similarity Score ($\mathbf{S_{Avg}}$)**

This is the final overall score, calculated as the simple average of the **Average Grouping Quality ($A_G$)** and the **Average Separation Quality ($A_S$)**.

| **Component**                            | **Definition**                                                                   | **Formula**                                                                  |
| :--------------------------------------- | :------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- |
| **Jaccard-Based Average Similarity Score ($\mathbf{S_{Avg}}$)** | The balanced overall score, averaging the quality of grouping and separation.                                             | $$\mathbf{S_{Avg}} = \frac{\mathbf{A_G} + \mathbf{A_S}}{2}$$ |

**Key Feature**: This metric treats the grouping decisions ($\mathbf{P}_{\text{Same}}$) and separation decisions ($\mathbf{P}_{\text{Sep}}$) as equally important, irrespective of their sample size ($\mathbf{N}_{\text{Same}}$ vs. $\mathbf{N}_{\text{Sep}}$). It directly gives the average reward earned per grouping decision and per separation decision, then averages those two averages.

In [24]:
# 'a' and 'b' values - sum of weights based on Jaccard similarity
a = 1402211.4217309589  # Quality of Grouping - total sum
b = 5164696351.643766   # Quality of Separation - total sum

# Counts - number of pairs fitting each category
same_pair_count = 20370689 + 590235997       # TP + FP - pairs in same predicted community
dif_pair_count = 5151393729 + 14028525       # TN + FN - pairs in different predicted communities

# Calculate ratios
a_ratio = a / same_pair_count
print(f"Average Grouping Quality: {a_ratio:4f}")
b_ratio = b / dif_pair_count
print(f"Average Separation Quality: {b_ratio:4f}")

overall_ratio = (a_ratio + b_ratio) / 2
print(f"Overall Quality Ratio: {overall_ratio:4f}")

Average Grouping Quality: 0.002296
Average Separation Quality: 0.999859
Overall Quality Ratio: 0.501078


- Quality of Grouping (a): 1,402,211.4
- Quality of Separation (b): 5,164,696,351.6

- Total weighted pairs: 5,166,098,563.1

- Total possible pairs: 5,776,028,940

- Jaccard Similarity Score: 0.8944
